<a href="https://colab.research.google.com/github/Amanda9805/Detecting-Machine-Generated-Texts/blob/train-model/COS_760_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers datasets fsspec evaluate

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import evaluate
import torch

from huggingface_hub import login
from datasets import load_dataset, Dataset, ClassLabel
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback)
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [ ]:
#shona_data = load_dataset("DigitalUmuganda/AfriVoice", "sn", streaming=True, split="train[:10%]")

# english_data = load_dataset("oscar-corpus/OSCAR-2201", language="en", streaming=True)

# num_samples = 500
# samples = []
# for i, example in enumerate(english_data["train"]):
#     if i >= num_samples:
#         break
#     samples.append(example['text'])

# eng_df = pd.DataFrame(samples, columns=['text'])
# eng_df.head()

zulu_data = load_dataset("dsfsi/vukuzenzele-monolingual", "zul")

def reformatData(dataDict):
  langList = []
  for index, row in dataDict.iterrows():
    # Access data using column names from the row Series
    if 'text' in row and row['text']:
        langList.append({
        'text': row.get('text', ''),
        'author': row.get('author', ''),
        'title': row.get('title', ''),
        'language': 'zul'
              })

  return langList

human_data = reformatData(zulu_data['train'].to_pandas())

zul_df = pd.DataFrame(human_data)
zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df['label'] = 0
zul_df.head()

In [ ]:
# eng_df['cleaned_text'] = eng_df['text'].apply(clean_text)
# eng_df['tokens'] = eng_df['cleaned_text'].apply(word_tokenize)
# print(eng_df.head())

zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df.head()

In [ ]:
from transformers import pipeline
import json

en_generator = pipeline('text-generation', model='gpt2')


english_prompts = [
    "Explain the significance of lobola in Southern Africa",
    "Write a short dialogue between two friends in Johannesburg",
    "Describe linguistic features that make isiZulu agglutinative"
]

zulu_prompts = [
    "Chaza ngokubaluleka kwesiko lwelobola eNingizimu Afrika",
    "Bhala inkulumo emfushane phakathi kwabangani ababili eGoli",
    "Landela indaba yamaZulu ngokomlando",
    "Chaza ngamasiko amasha eZulu eskhathini samanje",
    "Bhala inganekwane ethi 'UNogwaja noFudu'"
]

# def generate_with_prompts(prompts, generator, language, samples_per_prompt=3):
#     data = []
#     for prompt in prompts:
#         for _ in range(samples_per_prompt):
#             output = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
#             data.append({
#                 'prompt': prompt,
#                 'text': output[0]['generated_text'],
#                 'label': 'machine',
#                 'language': language,
#                 'prompt_type': 'cultural' if "tsika" in prompt else 'linguistic'  # Tag for analysis
#             })
#     return pd.DataFrame(data)


# eng_mg_df = generate_with_prompts(english_prompts, en_generator, 'English')
# eng_mg_df['cleaned_text'] = eng_mg_df['text'].apply(clean_text)
# eng_mg_df['tokens'] = eng_mg_df['cleaned_text'].apply(word_tokenize)
# print(eng_mg_df['text'].iloc[0][:300])

def load_jsonl_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line: {e}")
                        continue
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

    return data

zul_mg_df = pd.DataFrame(load_jsonl_data('zulu_mg_text.json'))

machine_data =reformatData(zul_mg_df)
zul_mg_df = pd.DataFrame(machine_data)
zul_mg_df['text'] = zul_mg_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
zul_mg_df['language'] = 'zul'
zul_mg_df['cleaned_text'] = zul_mg_df['text'].apply(clean_text)
zul_mg_df['tokens'] = zul_mg_df['cleaned_text'].apply(word_tokenize)
zul_mg_df['label'] = 1

zul_mg_df.head()

COMBINE THE DATASETS
-------------------------

In [ ]:
# Combine both datasets, this will be bad for us if we have more than 70% differnce in data length
# Combine both datasets
all_texts = pd.concat([zul_df, zul_mg_df], ignore_index=True)
all_texts_shuffled = all_texts.sample(frac=1).reset_index(drop=True)
print(f"Total texts after combining: {len(all_texts)}")
print(f"Human-written texts: {len(zul_df)}")
print(f"Machine-generated texts: {len(zul_mg_df)}")

# Create a DataFrame for easier analysis
combined_df = pd.DataFrame(all_texts_shuffled)
combined_df.head()

In [ ]:
# Create a balanced dataset with equal samples from each class
# we can choose to use this of the combined one
def create_balanced_dataset(df, target_size_per_class=None):
    class_counts = df['label'].value_counts()
    min_class_size = class_counts.min()
    if target_size_per_class:
        sample_size = min(target_size_per_class, min_class_size)
    else:
        sample_size = min_class_size
    balanced_df = df.groupby('label').sample(n=sample_size, random_state=42)
    return balanced_df.reset_index(drop=True)

# Create balanced dataset, because we need the same number of samples for each class
balanced_df = create_balanced_dataset(combined_df)
print(f"\nBalanced dataset created with {len(balanced_df)} samples")
print(f"Label distribution in balanced dataset: \n{balanced_df['label'].value_counts()}")

# Save balanced dataset
balanced_output_path = 'balanced_zulu_texts.csv'
balanced_df.to_csv(balanced_output_path, index=False, encoding='utf-8')
print(f"Balanced dataset saved to: {balanced_output_path}")
balanced_df.head()

Model training section
----------------------

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load the balanced dataset
balanced_data_path = 'balanced_zulu_texts.csv'
df = pd.read_csv(balanced_data_path, encoding='utf-8')

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# Cast the 'label' column to ClassLabel
dataset = dataset.cast_column('label', ClassLabel(num_classes=2, names=['human', 'machine']))

# Split into train and validation sets
train_val_split = dataset.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

# Load AfroXLMR tokenizer
model_name = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Load AfroXLMR model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True
)

# Define metrics for evaluation
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1']
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='/afroxlmr_finetuned',
    run_name='/afroxlmr_finetune_zulu',  # too much warnings so better to use a custom run name
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),  # Enable mixed precision only if GPU is available
    report_to="none"
)

# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('/afroxlmr_finetuned/model')
tokenizer.save_pretrained('/afroxlmr_finetuned/tokenizer')

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")